In [61]:
import pandas as pd

In [62]:

df = pd.read_hdf("./22-08-10-retention_features.h5")
print(df.columns)

Index(['user_id', 'card_id', 'card_text', 'is_new_fact',
       'user_n_study_positive', 'user_n_study_negative', 'user_n_study_total',
       'card_n_study_positive', 'card_n_study_negative', 'card_n_study_total',
       'usercard_n_study_positive', 'usercard_n_study_negative',
       'usercard_n_study_total', 'acc_user', 'acc_card', 'acc_usercard',
       'usercard_delta', 'usercard_delta_previous',
       'usercard_previous_study_response', 'leitner_box', 'sm2_efactor',
       'sm2_interval', 'sm2_repetition', 'delta_to_leitner_scheduled_date',
       'delta_to_sm2_scheduled_date', 'repetition_model',
       'elapsed_milliseconds', 'correct_on_first_try', 'utc_datetime',
       'utc_date', 'response', 'record_id', 'deck_id', 'deck_name',
       'n_minutes_spent'],
      dtype='object')


In [63]:
df.head

<bound method NDFrame.head of        user_id card_id                                          card_text  \
0           46   26811  People of the steppes of central Asia, governe...   
1           46   39163   Depose Hassan, the last Dey ruler of this city.    
2           46   31494  The historian Bede was from, this kingdom cent...   
3           46   22848  Kara Mustafa's attempts to take Vienna were th...   
4           46   33260  Man overthrown by Francisco Pizarro, an Incan ...   
...        ...     ...                                                ...   
123138     404   21394  Titan whom Zeus bound to a rock for giving fir...   
123139     404   38135  Running for President again in 2012, Russian l...   
123140     404   30418  Identify this scandal named for an oil field i...   
123141     404   34616        Position held by two people named Montezuma   
123142     404   38453  Position of leadership by which Enrico Dandolo...   

        is_new_fact  user_n_study_positive  u

In [64]:
df_orig = df.copy()
# df.rename(columns = {'label':'response'}, inplace = True)

In [65]:
import uuid
# Adapted from 
# https://stackoverflow.com/questions/66366449/how-to-generate-8-digit-unique-identifier-to-replace-the-existing-one-in-python
original_ids = df['user_id'].unique()
new_ids = {cid: uuid.uuid4().int for cid in original_ids}
df['Anon Student Id'] = df['user_id'].map(new_ids)
# Make each session id a user id

In [66]:
# Note this gets rid of milliseconds, as it seems datashop only takes up to three decimal places while %f is six
# Only
# df['Time'] = df['utc_date'].apply(lambda x: str(x) + " 00:00")
# # df['Time'] = df['utc_date'].dt.strftime('%Y-%m-%d %H:%M:%S')
# df['Time'].head

In [67]:
# No need for level? Is level different from KC
# https://pslcdatashop.web.cmu.edu/help?page=importFormatTd
df['Level (Deck)'] = df['deck_id'].apply(lambda x:"Level (" + x + ")")
# df['Problem Name'] = df['card_text']

In [68]:
df['Problem Name'] = df['card_id']

In [69]:
# df['Action'] = "Answer Question" 

In [70]:
df['Condition Name'] = df['repetition_model']
df['Time'] = df['utc_datetime']
df['Problem Start Time'] = df['utc_datetime']
df["KC (Single-KC)"] = "Single KC"
df["KC (Unique-Step)"] = df['card_id']

In [71]:
df['Condition Type'] = "Repetition Model"

In [72]:
# df.drop(['user_id'], inplace=True, axis=1)

df["Condition Name 2"] = df['deck_name']
df["Condition Type 2"] = "Deck"
df["Outcome"] = df['response'].apply(lambda x: "CORRECT" if x else "INCORRECT")
# df['Step Name'] = df['usercard_n_study_total']
df['Step Name'] = "Flashcard"
# df['Session ID'] = df['index']
# df['Session ID'] = "1" # Having one whole session for each student might make more sense?
learnsphere_df = df[["Anon Student Id", "Time", "Level (Deck)", "Problem Name", "Step Name", "Outcome", "Condition Name", "Condition Type", "Problem Start Time", "KC (Single-KC)", "KC (Unique-Step)", "Condition Name 2", "Condition Type 2"]]
# TODO: Create session id


In [73]:
learnsphere_df.index.names = ["Row"]

In [74]:
learnsphere_df.to_csv("22-08-10-learnsphere_karl.txt", sep="\t")